# agentscope.formatter 

agentscope.formatter 模块是 AgentScope 框架中用于将内部消息对象（Msg）格式化为符合各大 LLM API（如 OpenAI、DashScope、Anthropic、Gemini 等）输入格式的关键组件。其核心目标是： 

- 将统一的 Msg 对象（可能包含文本、图像、工具调用等）转换为各模型 API 所需的字典列表；
- 支持多模态（vision/audio/video）；
- 支持工具调用（function/tool calling）；
- 支持多智能体对话（multi-agent）场景；
- 可选支持按 token 数截断（truncation）。

主要 Formatter 类
1. OpenAIChatFormatter - 适用于 OpenAI GPT 系列
2. DashScopeChatFormatter - 适用于阿里云通义千问
3. AnthropicChatFormatter - 适用于 Anthropic Claude
4. GeminiChatFormatter - 适用于 Google Gemini
5. OllamaChatFormatter - 适用于 Ollama 本地模型

In [11]:
import asyncio
from agentscope.message import Msg, ToolUseBlock, ToolResultBlock
from agentscope.formatter import DashScopeChatFormatter

# 修正工具调用示例
tool_use = ToolUseBlock(
    tool_name="get_weather",
    args={"location": "Beijing"}
)

tool_result = ToolResultBlock(
    content="Sunny, 25°C",
    tool_name="get_weather"
)

# 修正：tool result 的 role 应该是 "tool"
msgs = [
    Msg(name="user", content="What's the weather in Beijing?", role="user"),
    Msg(name="assistant", content=[tool_use], role="assistant"),
    Msg(name="tool", content=[tool_result], role="assistant"),  # 修正这里
]

formatter = DashScopeChatFormatter()

formatted = await formatter.format(msgs)
print("修正后的工具调用格式化结果:")
for i, msg in enumerate(formatted):
    print(f"消息 {i+1}: {msg}")
    print("-" * 50)

2025-10-02 23:01:00,957 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.
2025-10-02 23:01:00,958 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.


修正后的工具调用格式化结果:
消息 1: {'role': 'user', 'content': "What's the weather in Beijing?"}
--------------------------------------------------


In [12]:
async def basic_text_example():
    """基本文本对话 - DashScope 格式"""
    msgs = [
        Msg(name="user", content="你好，请介绍一下通义千问", role="user"),
        Msg(name="assistant", content="通义千问是阿里云开发的大语言模型，具有强大的中文理解和生成能力。", role="assistant"),
        Msg(name="user", content="它有哪些主要特点？", role="user")
    ]
    
    formatter = DashScopeChatFormatter()
    formatted = await formatter.format(msgs)
    
    print("=== 基本文本对话 - DashScope 格式 ===")
    for i, msg in enumerate(formatted):
        print(f"消息 {i+1}: {msg}")
    print()

await basic_text_example()

=== 基本文本对话 - DashScope 格式 ===
消息 1: {'role': 'user', 'content': '你好，请介绍一下通义千问'}
消息 2: {'role': 'assistant', 'content': '通义千问是阿里云开发的大语言模型，具有强大的中文理解和生成能力。'}
消息 3: {'role': 'user', 'content': '它有哪些主要特点？'}



In [13]:
async def system_message_example():
    """系统消息示例"""
    msgs = [
        Msg(name="system", content="你是一个专业的Python编程助手，请用简洁明了的语言回答问题。", role="system"),
        Msg(name="user", content="如何在Python中创建一个列表？", role="user"),
        Msg(name="assistant", content="在Python中，可以使用方括号[]创建列表，例如：my_list = [1, 2, 3]", role="assistant")
    ]
    
    formatter = DashScopeChatFormatter()
    formatted = await formatter.format(msgs)
    
    print("=== 系统消息示例 - DashScope 格式 ===")
    for msg in formatted:
        print(msg)
    print()

await system_message_example()

=== 系统消息示例 - DashScope 格式 ===
{'role': 'system', 'content': '你是一个专业的Python编程助手，请用简洁明了的语言回答问题。'}
{'role': 'user', 'content': '如何在Python中创建一个列表？'}
{'role': 'assistant', 'content': '在Python中，可以使用方括号[]创建列表，例如：my_list = [1, 2, 3]'}



In [14]:
# 3. 多轮对话示例
async def multi_turn_conversation():
    """多轮对话示例"""
    msgs = [
        Msg(name="user", content="请帮我写一个计算斐波那契数列的函数", role="user"),
        Msg(name="assistant", content="好的，我来为您写一个斐波那契数列函数：\n\n```python\ndef fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)\n```", role="assistant"),
        Msg(name="user", content="这个函数有什么优化方法吗？", role="user"),
        Msg(name="assistant", content="是的，这个递归版本效率较低。可以使用动态规划优化：\n\n```python\ndef fibonacci_dp(n):\n    if n <= 1:\n        return n\n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n```", role="assistant"),
        Msg(name="user", content="谢谢，能解释一下时间复杂度的差异吗？", role="user")
    ]
    
    formatter = DashScopeChatFormatter()
    formatted = await formatter.format(msgs)
    
    print("=== 多轮对话示例 - DashScope 格式 ===")
    print(f"总共 {len(formatted)} 条消息")
    for i, msg in enumerate(formatted):
        print(f"第 {i+1} 条: role={msg['role']}, content长度={len(msg['content'])}")
    print()

await multi_turn_conversation()

=== 多轮对话示例 - DashScope 格式 ===
总共 5 条消息
第 1 条: role=user, content长度=17
第 2 条: role=assistant, content长度=127
第 3 条: role=user, content长度=13
第 4 条: role=assistant, content长度=176
第 5 条: role=user, content长度=18



In [17]:
# 4. 复杂工具调用示例
async def complex_tool_example():
    """复杂工具调用示例"""
    # 第一个工具调用
    search_tool = ToolUseBlock(
        tool_name="web_search",
        args={"query": "Python 最新版本特性", "count": 5}
    )
    
    search_result = ToolResultBlock(
        content="Python 3.12 引入了新的类型注解语法、性能优化等特性",
        tool_name="web_search"
    )
    
    # 第二个工具调用
    code_tool = ToolUseBlock(
        tool_name="code_generator",
        args={"language": "python", "description": "展示Python 3.12新特性"}
    )
    
    code_result = ToolResultBlock(
        content="# Python 3.12 新特性示例\ntype Point = tuple[float, float]  # 新的类型别名语法",
        tool_name="code_generator"
    )
    
    msgs = [
        Msg(name="user", content="请搜索Python最新版本的特性，并生成示例代码", role="user"),
        Msg(name="assistant", content=[search_tool], role="assistant"),
        Msg(name="tool", content=[search_result], role="assistant"),
        Msg(name="assistant", content=[code_tool], role="assistant"),
        Msg(name="tool", content=[code_result], role="assistant"),
        Msg(name="assistant", content="根据搜索结果，我为您生成了Python 3.12的新特性示例代码。", role="assistant")
    ]
    
    formatter = DashScopeChatFormatter()
    formatted = await formatter.format(msgs)
    
    print("=== 复杂工具调用示例 - DashScope 格式 ===")
    for i, msg in enumerate(formatted):
        print(f"消息 {i+1}:")
        print(f"  role: {msg['role']}")
        if 'content' in msg:
            content = msg['content']
            if isinstance(content, str):
                print(f"  content: {content[:100]}{'...' if len(content) > 100 else ''}")
            else:
                print(f"  content: {content}")
        if 'tool_calls' in msg:
            print(f"  tool_calls: {msg['tool_calls']}")
        print()
await complex_tool_example()

2025-10-02 23:03:21,199 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.
2025-10-02 23:03:21,201 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.
2025-10-02 23:03:21,201 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.
2025-10-02 23:03:21,202 | WARNING | _dashscope_formatter:_format:206 - Unsupported block type None in the message, skipped.


=== 复杂工具调用示例 - DashScope 格式 ===
消息 1:
  role: user
  content: 请搜索Python最新版本的特性，并生成示例代码

消息 2:
  role: assistant
  content: 根据搜索结果，我为您生成了Python 3.12的新特性示例代码。



In [18]:
# 5. 错误处理示例
async def error_handling_example():
    """错误处理示例"""
    try:
        # 测试空消息列表
        empty_msgs = []
        formatter = DashScopeChatFormatter()
        formatted = await formatter.format(empty_msgs)
        print("空消息列表格式化结果:", formatted)
        
        # 测试包含无效角色的消息
        invalid_msgs = [
            Msg(name="invalid_role", content="测试无效角色", role="invalid")
        ]
        formatted = await formatter.format(invalid_msgs)
        print("无效角色消息格式化结果:", formatted)
        
    except Exception as e:
        print(f"格式化过程中出现错误: {e}")

await error_handling_example()

空消息列表格式化结果: []
格式化过程中出现错误: 
